In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [2]:
# Define Directories
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Publish/Storm_Tracking/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Publish/Large_Scale_Data/'
#Output_Diri = '/glade/derecho/scratch/whimkao/Large_Scale_Data/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri, Subset):
    Data_DF = Open_File(Diri+Model+'_Data_'+Subset+'_Output.csv')
    ET_DF = Open_File(Diri+Model+'_ET_'+Subset+'_Output.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output.csv')
# Edit Time Format
    Time, Begin_Time, Compl_Time, Trop_Peak_Time, Peak_Time, Birth_Time = [], [], [], [], [], []
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Trop_Peak_Time.append(Datetime(ET_DF["Trop Peak Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
        Birth_Time.append(Datetime(ET_DF["Genesis Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Trop Peak Time"] = Trop_Peak_Time
    ET_DF["Peak Time"] = Peak_Time
    ET_DF["Genesis Time"] = Birth_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
# Edit Time Format
def Datetime(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    except:
        New_Time = numpy.nan
    return (New_Time)

In [6]:
# Create Phase Distribution Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)
Lat_Bins = Create_Bins(10,60,5)
Lon_Bins = Create_Bins(-100,20,5)

In [7]:
# Number of Years For Each Climate Scenario
Num_Years = numpy.array([90,93,93])

In [8]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [9]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [10]:
# Open Files
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri, "SubsetB")
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri, "SubsetB")
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri, "SubsetB")

In [11]:
# Create List of Monthly File Names
def Monthly_File_Names(Model, Years, Months, File_Names):
    for j in range(12):
        for i in range(len(Years)):
            File_Name = Model + '/' + Model + '.cam.h0.' + str(Years[i]) + '-' + str(Months[j]) + '.nc_regrid.nc'
            File_Names[j].append(File_Name)
    return (File_Names)

In [12]:
# List of Years For Each Climate Scenario
Control_Years = Create_Bins(1985,2014,1)
RCP45_Years = Create_Bins(2069,2099,1)
RCP85_Years = Create_Bins(2069,2099,1)
Months = ['01','02','03','04','05','06','07','08','09','10','11','12']
Months_Name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [13]:
# Define File Names For Control
Control_Monthly_Files = []
for j in range(12):
    Control_Monthly_Files.append([])
Control_Monthly_Files = Monthly_File_Names('CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002', Control_Years, Months, Control_Monthly_Files)
Control_Monthly_Files = Monthly_File_Names('CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004', Control_Years, Months, Control_Monthly_Files)
Control_Monthly_Files = Monthly_File_Names('CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900', Control_Years, Months, Control_Monthly_Files)

In [14]:
# Define File Names For RCP4.5
RCP45_Monthly_Files = []
for j in range(12):
    RCP45_Monthly_Files.append([])
RCP45_Monthly_Files = Monthly_File_Names('CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900', RCP45_Years, Months, RCP45_Monthly_Files)
RCP45_Monthly_Files = Monthly_File_Names('CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002', RCP45_Years, Months, RCP45_Monthly_Files)
RCP45_Monthly_Files = Monthly_File_Names('CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003', RCP45_Years, Months, RCP45_Monthly_Files)

In [15]:
# Define File Names For RCP8.5
RCP85_Monthly_Files = []
for j in range(12):
    RCP85_Monthly_Files.append([])
RCP85_Monthly_Files = Monthly_File_Names('CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900', RCP85_Years, Months, RCP85_Monthly_Files)
RCP85_Monthly_Files = Monthly_File_Names('CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003', RCP85_Years, Months, RCP85_Monthly_Files)
RCP85_Monthly_Files = Monthly_File_Names('CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004', RCP85_Years, Months, RCP85_Monthly_Files)

In [16]:
# Create Function to Open Monthly Files
def Open_Monthly_File(File_Names, Vars):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/'
    for i in range(len(File_Names)):
        if Vars == "All":
            Month_DS = xr.open_dataset(Diri + File_Names[i])
        else:
            Month_DS = xr.open_dataset(Diri + File_Names[i])[Vars]
        Month_DS_New = Month_DS.where((Month_DS.lon >= -100) & (Month_DS.lon <= 0) & \
        (Month_DS.lat >= 0) & (Month_DS.lat <= 60), drop=True)
# Edit Time Stamp To Make Sure Dates Don't Overlap
        if i < (len(File_Names) / 3):
            Month_DS_New['time'] = Month_DS_New['time'] - numpy.timedelta64(16,'D')
        elif i < (len(File_Names) / 3 * 2):
            Month_DS_New['time'] = Month_DS_New['time'] - numpy.timedelta64(10,'D')
        else:
            Month_DS_New['time'] = Month_DS_New['time'] - numpy.timedelta64(4,'D')
# Combine Datasets
        if i == 0:
            Month_DS_Combine = Month_DS_New
        else:
            Month_DS_Combine = xr.concat([Month_DS_Combine, Month_DS_New], dim='time')
        if i % 10 == 8:
            print (i)
# Calculate Mean Values Across Time
    Month_DS_Mean = Month_DS_Combine.mean(dim='time')
    return (Month_DS_Mean)

In [17]:
# Define Which Variables We Will Need
Vars = ['T', 'TS', 'Z3', 'PSL', 'U', 'V', 'FLUT', 'TMQ', 'OMEGA', 'OCNFRAC']

In [18]:
# Open Files For Control (Jan - Mar)
Control_DS_Jan = Open_Monthly_File(Control_Monthly_Files[0], Vars)
Control_DS_Feb = Open_Monthly_File(Control_Monthly_Files[1], Vars)
Control_DS_Mar = Open_Monthly_File(Control_Monthly_Files[2], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [19]:
# Open Files For Control (Apr - Jun)
Control_DS_Apr = Open_Monthly_File(Control_Monthly_Files[3], Vars)
Control_DS_May = Open_Monthly_File(Control_Monthly_Files[4], Vars)
Control_DS_Jun = Open_Monthly_File(Control_Monthly_Files[5], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [20]:
# Open Files For Control (Jul - Sep)
Control_DS_Jul = Open_Monthly_File(Control_Monthly_Files[6], Vars)
Control_DS_Aug = Open_Monthly_File(Control_Monthly_Files[7], Vars)
Control_DS_Sep = Open_Monthly_File(Control_Monthly_Files[8], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [21]:
# Open Files For Control (Oct - Dec)
Control_DS_Oct = Open_Monthly_File(Control_Monthly_Files[9], Vars)
Control_DS_Nov = Open_Monthly_File(Control_Monthly_Files[10], Vars)
Control_DS_Dec = Open_Monthly_File(Control_Monthly_Files[11], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [22]:
# Open Files For RCP4.5 (Jan - Mar)
RCP45_DS_Jan = Open_Monthly_File(RCP45_Monthly_Files[0], Vars)
RCP45_DS_Feb = Open_Monthly_File(RCP45_Monthly_Files[1], Vars)
RCP45_DS_Mar = Open_Monthly_File(RCP45_Monthly_Files[2], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [23]:
# Open Files For RCP4.5 (Apr - Jun)
RCP45_DS_Apr = Open_Monthly_File(RCP45_Monthly_Files[3], Vars)
RCP45_DS_May = Open_Monthly_File(RCP45_Monthly_Files[4], Vars)
RCP45_DS_Jun = Open_Monthly_File(RCP45_Monthly_Files[5], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [24]:
# Open Files For RCP4.5 (Jul - Sep)
RCP45_DS_Jul = Open_Monthly_File(RCP45_Monthly_Files[6], Vars)
RCP45_DS_Aug = Open_Monthly_File(RCP45_Monthly_Files[7], Vars)
RCP45_DS_Sep = Open_Monthly_File(RCP45_Monthly_Files[8], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [25]:
# Open Files For RCP4.5 (Oct - Dec)
RCP45_DS_Oct = Open_Monthly_File(RCP45_Monthly_Files[9], Vars)
RCP45_DS_Nov = Open_Monthly_File(RCP45_Monthly_Files[10], Vars)
RCP45_DS_Dec = Open_Monthly_File(RCP45_Monthly_Files[11], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [26]:
# Open Files For RCP8.5 (Jan - Mar)
RCP85_DS_Jan = Open_Monthly_File(RCP85_Monthly_Files[0], Vars)
RCP85_DS_Feb = Open_Monthly_File(RCP85_Monthly_Files[1], Vars)
RCP85_DS_Mar = Open_Monthly_File(RCP85_Monthly_Files[2], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [27]:
# Open Files For RCP8.5 (Apr - Jun)
RCP85_DS_Apr = Open_Monthly_File(RCP85_Monthly_Files[3], Vars)
RCP85_DS_May = Open_Monthly_File(RCP85_Monthly_Files[4], Vars)
RCP85_DS_Jun = Open_Monthly_File(RCP85_Monthly_Files[5], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [28]:
# Open Files For RCP8.5 (Jul - Sep)
RCP85_DS_Jul = Open_Monthly_File(RCP85_Monthly_Files[6], Vars)
RCP85_DS_Aug = Open_Monthly_File(RCP85_Monthly_Files[7], Vars)
RCP85_DS_Sep = Open_Monthly_File(RCP85_Monthly_Files[8], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [29]:
# Open Files For RCP8.5 (Oct - Dec)
RCP85_DS_Oct = Open_Monthly_File(RCP85_Monthly_Files[9], Vars)
RCP85_DS_Nov = Open_Monthly_File(RCP85_Monthly_Files[10], Vars)
RCP85_DS_Dec = Open_Monthly_File(RCP85_Monthly_Files[11], Vars)

8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88
8
18
28
38
48
58
68
78
88


In [30]:
# Create Dictionaries to Store Datasets
Control_DS_Dict = {"Jan": Control_DS_Jan, "Feb": Control_DS_Feb, "Mar": Control_DS_Mar, \
"Apr": Control_DS_Apr, "May": Control_DS_May, "Jun": Control_DS_Jun, \
"Jul": Control_DS_Jul, "Aug": Control_DS_Aug, "Sep": Control_DS_Sep, \
"Oct": Control_DS_Oct, "Nov": Control_DS_Nov, "Dec": Control_DS_Dec}
RCP45_DS_Dict = {"Jan": RCP45_DS_Jan, "Feb": RCP45_DS_Feb, "Mar": RCP45_DS_Mar, \
"Apr": RCP45_DS_Apr, "May": RCP45_DS_May, "Jun": RCP45_DS_Jun, \
"Jul": RCP45_DS_Jul, "Aug": RCP45_DS_Aug, "Sep": RCP45_DS_Sep, \
"Oct": RCP45_DS_Oct, "Nov": RCP45_DS_Nov, "Dec": RCP45_DS_Dec}
RCP85_DS_Dict = {"Jan": RCP85_DS_Jan, "Feb": RCP85_DS_Feb, "Mar": RCP85_DS_Mar, \
"Apr": RCP85_DS_Apr, "May": RCP85_DS_May, "Jun": RCP85_DS_Jun, \
"Jul": RCP85_DS_Jul, "Aug": RCP85_DS_Aug, "Sep": RCP85_DS_Sep, \
"Oct": RCP85_DS_Oct, "Nov": RCP85_DS_Nov, "Dec": RCP85_DS_Dec}

In [31]:
# Combine Monthly Files Based on Seasons (Summer, Fall, Winter-Spring)
def Combine_Monthly_Files(DS_Dict):
    DS_Dict['Jun-Aug'] = xr.concat([DS_Dict['Jun'], DS_Dict['Jul'], DS_Dict['Aug']], dim='time').mean(dim='time')
    print ("1")
    DS_Dict['Sep-Nov'] = xr.concat([DS_Dict['Sep'], DS_Dict['Oct'], DS_Dict['Nov']], dim='time').mean(dim='time')
    print ("2")
    DS_Dict['Dec-May'] = xr.concat([DS_Dict['Dec'], DS_Dict['Jan'], DS_Dict['Feb'], \
    DS_Dict['Mar'], DS_Dict['Apr'], DS_Dict['May']], dim='time').mean(dim='time')
    print ("3")
    DS_Dict['Annual'] = xr.concat([DS_Dict['Dec-May'], DS_Dict['Jun-Aug'], DS_Dict['Sep-Nov']], dim='time').mean(dim='time')
    print ("4")
    return (DS_Dict)

In [32]:
# Combine Monthly Files For Control
Control_DS_Dict = Combine_Monthly_Files(Control_DS_Dict)

1
2
3
4


In [33]:
# Combine Monthly Files For RCP4.5
RCP45_DS_Dict = Combine_Monthly_Files(RCP45_DS_Dict)

1
2
3
4


In [34]:
# Combine Monthly Files For RCP8.5
RCP85_DS_Dict = Combine_Monthly_Files(RCP85_DS_Dict)

1
2
3
4


In [35]:
# Months Name
Months_Name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jun-Aug', 'Sep-Nov', 'Dec-May', 'Annual']

In [36]:
# Output DS to netCDF File
def Output_DS(DS, Model, Month):
    File_Name = str(Model+'_Output_'+Month+'.nc')
    DS.to_netcdf(Output_Diri+File_Name)

In [37]:
# Output Control Files For Each Month
for i in range(16):
    DS = Control_DS_Dict[Months_Name[i]]
    Output_DS(DS, "Control", Months_Name[i])
    print (i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [38]:
# Output RCP4.5 Files For Each Month
for i in range(16):
    DS = RCP45_DS_Dict[Months_Name[i]]
    Output_DS(DS, "RCP45", Months_Name[i])
    print (i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [39]:
# Output RCP8.5 Files For Each Month
for i in range(16):
    DS = RCP85_DS_Dict[Months_Name[i]]
    Output_DS(DS, "RCP85", Months_Name[i])
    print (i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [40]:
Output_DS(Control_DS_Jan, 'Control', 'Jan')